## pix2pix pytorch lightning

### Data loader

In [ ]:
import os
from argparse import ArgumentParser, Namespace
from collections import OrderedDict
from pytorch_lightning.trainer import Trainer
from models.pix2pix_model import Pix2PixModel

In [ ]:
model = Pix2PixModel()

### Train

In [ ]:
trainer = Trainer(gpus=[0], max_epochs=100)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type          | Params
------------------------------------------------------
0 | generator           | GeneratorUNet | 54 M  
1 | discriminator       | Discriminator | 2 M   
2 | criterion_GAN       | MSELoss       | 0     
3 | criterion_pixelwise | L1Loss        | 0     
/home/nghia/miniconda3/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/home/nghia/miniconda3/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in t